# Create edge shapefiles

- Input: node shapefiles 
- Output: edge shapefiles

The edge shapefiles are used for visualizing networks, not in the modeling process.


In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
import pickle
import copy
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from sklearn.linear_model import LinearRegression
from shapely.geometry import LineString


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append("../")
import utils

In [4]:
import importlib
importlib.reload(utils)

<module 'utils' from '../utils.py'>

In [13]:
# # read files
# with open("../../data/02_intermediate/greater_boston_ct_socioecon_shp.pickle", 'rb') as f:
#     df_shp = pickle.load(f)


In [5]:
# read files
with open("../../data/02_intermediate/boston_miami_chicago_ct_shp_dic.pickle", 'rb') as f:
    shp_dic = pickle.load(f)


In [6]:
shp_dic.keys()

dict_keys(['boston', 'miami', 'chicago'])

In [7]:
# create edge_dic
edge_dic = {}
city_list = ['boston', 'chicago', 'miami']
origin_col_name = 'GEOID_home'
destination_col_name = 'GEOID'
for city in city_list:
    edge_dic[city]=utils.turn_node_to_edge_shp(shp_dic[city], origin_col_name, destination_col_name)


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000

In [8]:
# add distance and change index names for the three edge shapefiles.
for city in edge_dic.keys():
    edge_dic[city]['distance'] = edge_dic[city].geometry.length
    edge_dic[city].index = list(tuple(zip(edge_dic[city].GEOID_home, edge_dic[city].GEOID)))
    

/home/jtl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
# # Turn the node to edge shapefile
# origin_col_name = 'GEOID_home'
# destination_col_name = 'GEOID'
# edge_shp = utils.turn_node_to_edge_shp(df_shp, origin_col_name, destination_col_name)


In [17]:
# edge_shp['distance'] = edge_shp.geometry.length

/home/jtl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [27]:
# edge_shp.index = list(tuple(zip(edge_shp.GEOID_home, edge_shp.GEOID)))

In [10]:
edge_dic['boston']

,GEOID_home,GEOID,geometry,distance
"(25025170501, 25025170501)",25025170501,25025170501,"LINESTRING (-70.98928 42.41592, -70.98928 42.4...",0.000000
"(25025170501, 25021418004)",25025170501,25021418004,"LINESTRING (-70.98928 42.41592, -71.01050 42.2...",0.178808
"(25025170501, 25021418102)",25025170501,25021418102,"LINESTRING (-70.98928 42.41592, -71.02163 42.2...",0.172228
"(25025170501, 25021417702)",25025170501,25021417702,"LINESTRING (-70.98928 42.41592, -70.99154 42.2...",0.156114
"(25025170501, 25021417701)",25025170501,25021417701,"LINESTRING (-70.98928 42.41592, -71.00118 42.2...",0.164010
...,...,...,...,...
"(33015065008, 33017080202)",33015065008,33017080202,"LINESTRING (-70.82785 42.91395, -70.89611 43.1...",0.234102
"(33015065008, 33017080100)",33015065008,33017080100,"LINESTRING (-70.82785 42.91395, -70.91886 43.1...",0.213939
"(33015065008, 33015107400)",33015065008,33015107400,"LINESTRING (-70.82785 42.91395, -70.80298 43.0...",0.150983
"(33015065008, 33015063002)",33015065008,33015063002,"LINESTRING (-70.82785 42.91395, -70.84414 42.8...",0.034444


# Save

In [11]:
# save
with open('../../data/02_intermediate/edge_shp_dic.pickle', 'wb') as f:
    pickle.dump(edge_dic, f)
